In [1]:
# This work is inspired by https://github.com/matterport/Mask_RCNN

# Data download and unzipping
!wget https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_test.zip -c
!wget https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_train.zip -c
# !wget https://raw.githubusercontent.com/ZengChen94/Kaggle-Dataset/master/stage1_test.zip -c
# !wget https://raw.githubusercontent.com/ZengChen94/Kaggle-Dataset/master/stage1_train.zip -c
!mkdir stage1_train stage1_test
!unzip -q stage1_train.zip -d stage1_train/
!unzip -q stage1_test.zip -d stage1_test/

# Downloading and intstalling keras
!pip install keras
!pip install tqdm
!pip install imgaug

In [2]:
# Downloading the library Mask_RCNN
!git clone https://github.com/matterport/Mask_RCNN.git
# !git clone https://github.com/ZengChen94/Mask_RCNN.git
!mv -r ./Mask_RCNN/* ./
!rm -r ./Mask_RCNN
!ls

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline 

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
# import utils
import cv2
from glob import glob
import skimage.io
import seaborn as sns
import sys
import math
import re
import time
import matplotlib
import matplotlib.patches as patches
# from config import Config
# import model as modellib
# import visualize
# from model import log
import imgaug
import warnings

from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

warnings.simplefilter(action='ignore', category=FutureWarning)

ROOT_DIR = os.getcwd()
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)
TRAIN_PATH = './stage1_train/'
TEST_PATH = './stage2_test/'
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]
kernel = np.ones((1,1),np.uint8)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
class KaggleTrainDataset(utils.Dataset):
    def get_path(self, id_):
        return './stage1_train/' + id_ + '/images/' + id_ + '.png'
    
    def load_data(self, train_ids):
        self.add_class("nuclei", 1, "nuclei")
        self.sizes = []
        self.img_ids = train_ids
        for i, id_ in tqdm(enumerate(self.img_ids), total=len(self.img_ids)):
            path = self.get_path(id_)
            img = imread(path)
            if (len(img.shape) == 2):
                img2 = np.zeros((img.shape[0], img.shape[1], 3))
                img2[:,:,0] = img
                img2[:,:,1] = img
                img2[:,:,2] = img
                img = img2
            img = img[:, :, :3]
            img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
            w, h = img.shape[0], img.shape[1]
            self.sizes.append((w, h))
            self.add_image("nuclei", image_id = i, path=id_, width=w, height=h)

    def load_image(self, image_id):
        image_name = self.img_ids[image_id]
        path = self.get_path(image_name)
        img = imread(path)
        if (len(img.shape) == 2):
            img2 = np.zeros((img.shape[0], img.shape[1], 3))
            img2[:,:,0] = img
            img2[:,:,1] = img
            img2[:,:,2] = img
            img = img2
        img = img[:, :, :3]
        w, h = img.shape[0], img.shape[1]
        return img
    
    def load_mask(self, image_id):
        id_ = self.img_ids[image_id]
        mask_path = './stage1_train/' + id_ + '/masks/'
        num_masks = len(next(os.walk(mask_path))[2])
        width, height = self.sizes[image_id]
        mask = np.zeros((width, height, num_masks), dtype=np.bool)
        for i, mask_file in enumerate(next(os.walk(mask_path))[2]):
            mask_ = imread(mask_path + mask_file)
            mask_ = mask_.reshape((mask_.shape[0], mask_.shape[1], 1))
            mask[:,:,i:i+1] = mask_
        return mask, np.ones(num_masks, np.int32)
      
class KaggleTestDataset(utils.Dataset):
    def get_path(self, id_):
        return './stage2_test/' + id_ + '/images/' + id_ + '.png'
    
    def load_data(self, test_ids):
        self.add_class("nuclei", 1, "nuclei")
        self.sizes = []
        self.img_ids = test_ids
        for i, id_ in tqdm(enumerate(self.img_ids), total=len(self.img_ids)):
            path = self.get_path(id_)
            img = imread(path)
            if (len(img.shape) == 2):
                img2 = np.zeros((img.shape[0], img.shape[1], 3))
                img2[:,:,0] = img
                img2[:,:,1] = img
                img2[:,:,2] = img
                img = img2
            img = img[:, :, :3]
            img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
            w, h = img.shape[0], img.shape[1]
            self.sizes.append((w, h))
            self.add_image("nuclei", image_id = i, path=id_, width=w, height=h)

    def load_image(self, image_id):
        image_name = self.img_ids[image_id]
        path = self.get_path(image_name)
        img = imread(path)
        if (len(img.shape) == 2):
            img2 = np.zeros((img.shape[0], img.shape[1], 3))
            img2[:,:,0] = img
            img2[:,:,1] = img
            img2[:,:,2] = img
            img = img2
        img = img[:, :, :3]
        w, h = img.shape[0], img.shape[1]
        return img

In [4]:
# Load dataset
dataset_train = KaggleTrainDataset()
dataset_train.load_data(train_ids[:600])
# dataset_train.load_data(train_ids[:1000])
dataset_train.prepare()

dataset_val = KaggleTrainDataset()
dataset_val.load_data(train_ids[600:664])
dataset_val.prepare()

dataset_test = KaggleTestDataset()
dataset_test.load_data(test_ids)
dataset_test.prepare()

100%|██████████| 3019/3019 [00:25<00:00, 117.83it/s]


In [5]:
# Configurations
class KaggleConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    NAME = "shapes"
    
#     GPU_COUNT = 1
#     IMAGES_PER_GPU = 8
#     NUM_CLASSES = 1 + 1
#     IMAGE_MIN_DIM = 512
#     IMAGE_MAX_DIM = 512
#     RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  
#     TRAIN_ROIS_PER_IMAGE = 32
#     STEPS_PER_EPOCH = 100
#     VALIDATION_STEPS = 5

    IMAGES_PER_GPU = 6
    NUM_CLASSES = 1 + 1  # Background + nucleus
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    IMAGE_MIN_SCALE = 2.0
    
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 100

    DETECTION_MIN_CONFIDENCE = 0
    BACKBONE = "resnet50"
    
    RPN_NMS_THRESHOLD = 0.9
    RPN_TRAIN_ANCHORS_PER_IMAGE = 64

    MEAN_PIXEL = np.array([43.53, 39.56, 48.22])

    USE_MINI_MASK = True
    MINI_MASK_SHAPE = (56, 56)
    TRAIN_ROIS_PER_IMAGE = 128
    MAX_GT_INSTANCES = 200
    DETECTION_MAX_INSTANCES = 400
    
config = KaggleConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     6
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 6
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                2.0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               200
MEAN_PIXEL                     [43.53 39.56 48.22]
MINI_MASK_SHAPE                (56, 56)
NAME                           shapes
NUM_CLASSES                    2
POOL_SIZE                      7
POST_NMS_

In [8]:
# Notebook Preferences
def get_ax(rows=1, cols=1, size=8):
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [9]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

# Weights to start with
init_with = "coco"  # imagenet, coco, or last
if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)

In [11]:
# # http://imgaug.readthedocs.io/en/latest/source/augmenters.html

# augmentation = imgaug.augmenters.Fliplr(0.5)

augmentation = imgaug.augmenters.SomeOf((0, 2), [
        imgaug.augmenters.Fliplr(0.5),
        imgaug.augmenters.Flipud(0.5),
        imgaug.augmenters.OneOf([imgaug.augmenters.Affine(rotate=90),
                   imgaug.augmenters.Affine(rotate=180),
                   imgaug.augmenters.Affine(rotate=270)]),
        imgaug.augmenters.Multiply((0.8, 1.5)),
        imgaug.augmenters.GaussianBlur(sigma=(0.0, 5.0))
    ])

model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=20,
                augmentation=augmentation,
                layers='heads')

model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=60,
                augmentation=augmentation,
                layers='all')

Instructions for updating:
Use the retry module or similar alternatives.

Starting at epoch 58. LR=0.001

Checkpoint Path: /home/ubuntu/kaggle-src/logs/shapes20180411T2031/mask_rcnn_shapes_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2 

/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'



Starting at epoch 58. LR=0.001

Checkpoint Path: /home/ubuntu/kaggle-src/logs/shapes20180411T2031/mask_rcnn_shapes_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         (Conv2D)
bn3a_br

/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:2330: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


100/100 [==============================] - 718s 7s/step - loss: 1.2362 - rpn_class_loss: 0.2569 - rpn_bbox_loss: 0.4560 - mrcnn_class_loss: 0.2279 - mrcnn_bbox_loss: 0.1062 - mrcnn_mask_loss: 0.1893 - val_loss: 0.9407 - val_rpn_class_loss: 0.0962 - val_rpn_bbox_loss: 0.2582 - val_mrcnn_class_loss: 0.2725 - val_mrcnn_bbox_loss: 0.1166 - val_mrcnn_mask_loss: 0.1972
Epoch 60/60
100/100 [==============================] - 552s 6s/step - loss: 0.8281 - rpn_class_loss: 0.0755 - rpn_bbox_loss: 0.2230 - mrcnn_class_loss: 0.2335 - mrcnn_bbox_loss: 0.1020 - mrcnn_mask_loss: 0.1941 - val_loss: 0.9372 - val_rpn_class_loss: 0.0846 - val_rpn_bbox_loss: 0.2315 - val_mrcnn_class_loss: 0.2767 - val_mrcnn_bbox_loss: 0.1304 - val_mrcnn_mask_loss: 0.2139


In [12]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes.h5")
model.keras_model.save_weights(model_path)

In [32]:
# Detection
class InferenceConfig(KaggleConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    RPN_NMS_THRESHOLD = 0.7

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()[1]
# model_path = './logs/shapes20180411T2031/mask_rcnn_shapes_0060.h5'

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

Loading weights from  /home/ubuntu/kaggle-src/logs/shapes20180411T2031/mask_rcnn_shapes_0060.h5


In [24]:
# # Test on a random image
# image_id = random.choice(dataset_val.image_ids)

# original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#     modellib.load_image_gt(dataset_val, inference_config, 
#                            image_id, use_mini_mask=False)

# # Ground truth
# log("original_image", original_image)
# log("image_meta", image_meta)
# log("gt_class_id", gt_class_id)
# log("gt_bbox", gt_bbox)
# log("gt_mask", gt_mask)

# visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
#                             dataset_train.class_names, figsize=(8, 8))

In [25]:
# results = model.detect([original_image], verbose=1)

# r = results[0]
# visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
#                             dataset_val.class_names, r['scores'], ax=get_ax())

# # r['masks'].shape = 256*256*num
# # plt.imshow(r['masks'][:,:,0])

In [26]:
# # Compute VOC-Style mAP @ IoU=0.5
# # Running on 10 images. Increase for better accuracy.
# image_ids = np.random.choice(dataset_val.image_ids, 10)
# APs = []
# for image_id in image_ids:
#     # Load image and ground truth data
#     image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#         modellib.load_image_gt(dataset_val, inference_config,
#                                image_id, use_mini_mask=False)
#     molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
#     # Run object detection
#     results = model.detect([image], verbose=0)
#     r = results[0]
#     # Compute AP
#     AP, precisions, recalls, overlaps =\
#         utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
#                          r["rois"], r["class_ids"], r["scores"], r['masks'])
#     APs.append(AP)
    
# print("mAP: ", np.mean(APs))

In [29]:
# rle
# inspired by the discussion of kaggle
def numpy2encoding_no_overlap2(predicts, img_id):
    sum_predicts = np.sum(predicts, axis=2)
    rows, cols = np.where(sum_predicts>=2)
    
    for i in zip(rows, cols):
        instance_indicies = np.where(np.any(predicts[i[0],i[1],:]))[0]
        highest = instance_indicies[0]
        predicts[i[0],i[1],:] = predicts[i[0],i[1],:]*0
        predicts[i[0],i[1],highest] = 1
    
    ImageId = []
    EncodedPixels = []
    for i in range(predicts.shape[2]): 
        rle = run_length_encoding(predicts[:,:,i])
        if len(rle)>0:
            ImageId.append(img_id)
            EncodedPixels.append(rle)
    if len(ImageId) == 0:
        ImageId.append(img_id)
        EncodedPixels.append('')
    return ImageId, EncodedPixels

def run_length_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    run_lengths = ' '.join([str(r) for r in run_lengths])
    return run_lengths

In [30]:
ImageId = []
EncodedPixels = []

for image_id in dataset_test.image_ids:
    img_id = dataset_test.img_ids[image_id]
    image = dataset_test.load_image(image_id)
    results = model.detect([image], verbose=0)
    r = results[0]
#     idx = utils.non_max_suppression(r['rois'], r['scores'], 0.2)
#     ImageId_batch, EncodedPixels_batch = numpy2encoding_no_overlap2(r['masks'][idx], img_id)
    ImageId_batch, EncodedPixels_batch = numpy2encoding_no_overlap2(r['masks'], img_id)
    ImageId += ImageId_batch
    EncodedPixels += EncodedPixels_batch
    
saved_file = 'rcnn-410.csv'

# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = ImageId
# sub['EncodedPixels'] = pd.Series(EncodedPixels).apply(lambda x: ''.join(str(y) for y in x))
sub['EncodedPixels'] = EncodedPixels
sub.to_csv(saved_file, index=False)